In [1]:
import pandas as pd
import requests
import warnings
import time
import multiprocessing
import threading
from concurrent.futures import ThreadPoolExecutor
# from google.colab import files
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [29]:
# uploaded = files.upload()
# df = pd.read_csv(io.BytesIO(uploaded[list(uploaded.keys())[0]]))
df = pd.read_csv('bangla_to_ipa_unique.csv')

In [3]:
def subset_worker_words(subset):
  print(subset)
  for row in subset.itertuples():
    print(f"Started {row.Index}")
    if pd.notna(row.ipa):
      print(f"{row.Index}:{row.ipa}")
      continue
    words=row.sentence.split(" ")
    if(len(words)>1):
      datas=""
      for word in words:
        # url= f"http://ipa.bangla.gov.bd/post_text_bangla_ipa?bangla_ipa={word}"
        url= f"http://127.0.0.1:5000//post_text_bangla_ipa?bangla_ipa={word}"
        response=None
        try:
          response = requests.get (url, verify=False)
          if response and response.status_code == 200:
            data= response.json()
            datas=datas+" "+data["data"][0]
          else:
            datas=datas+" "+"ERROR"
        except:
          print(f"error at{row.Index}:{row.sentence}")
          datas=datas+" "+"ERROR"
      print(f"Ended {row.Index}")
      df.at[row.Index,"ipa"]=datas
    else:
      url = f"http://ipa.bangla.gov.bd/post_text_bangla_ipa?bangla_ipa={row.sentence}"
      response=None
      try:
        response = requests.get (url, verify=False)
        if response and response.status_code == 200:
          data= response.json()
          df.at[row.Index,"ipa"]=data["data"][0]
          print(f"Ended {row.Index}:{row.sentence}")
          print(f"{row.Index}:{data['data'][0]}")
        else:
          data= None
      except:
        print(f"error at{row.Index}:{row.sentence}")
  # df.to_csv("withIPA.csv",index=False)
  print("Done")
  return

In [4]:
def subset_worker(subset):
  print(subset)
  for row in subset.itertuples():
    print(f"Started{row.Index}")
    if pd.notna(row.ipa):
      print(f"{row.Index}:{row.ipa}")
      continue
    url = f"http://127.0.0.1:5000//post_text_bangla_ipa?bangla_ipa={row.sentence}"
    response=None
    try:
      response = requests.get (url, verify=False)
      if response and response.status_code == 200:
        data= response.json()
        df.at[row.Index,"ipa"]=data["data"][0]
      else:
        data= None
    except:
      print(f"error at{row.Index}:{row.sentence}")
    print(f"Ended {row.Index}")
    # print(f"{row.Index}:{row.ipa}")
  # df.to_csv("withIPA.csv",index=False)
  print("Done")
  return

In [10]:
def subset_worker_db(subset):
  for row in subset.itertuples():
    data=call_api(row.word)
    if data:
      print(f"{row.Index}:{data['data'][0]}")
    else:
      print(row.Index)
      print(row.word)
      break

In [13]:
# threadpool make sure everything is currect before running this cell
start=30000
steps=1000
end=70000
subsets=[]
start_pos=start
while start<end:
  subsets.append(df.iloc[start:start+steps])
  start=start+steps
print(subsets)
pool=ThreadPoolExecutor(len(subsets))
ws=[]
for subset in subsets:
  ws.append(pool.submit(subset_worker_db, subset))
print(pool.shutdown())
df.to_csv(f"words{start_pos}_{end}.csv",index=False)

[             word           ipa
30000     হ্যানসন        hænʃɔn
30001    পক্ষাঘাত   pokkʰagʱat̪
30002      সিআরপি     si͡a̯rɔpi
30003       ফর্মে        pʰɔrme
30004    দেখাদেখি  d̪ækʰad̪ekʰi
...           ...           ...
30995     সেন্টার        sentar
30996       হায়ার         haʲar
30997     স্টাডিজ        stadiɟ
30998  রিসার্চ-এর    risarco-er
30999   ক্যাপ্টেন        kæpten

[1000 rows x 2 columns],                word             ipa
31000        সাইদুল       sa͡i̯d̪ul
31001             ই               i
31002        পিএসসি        pi͡e̯ssi
31003          বিএন          bi͡e̯n
31004         ৯৯০টি           990ti
...             ...             ...
31995  বৃদ্ধাঙ্গুলি  brid̪d̪ʱaŋguli
31996   নিরবচ্ছিন্ন   nirɔbɔccʰinno
31997       সররাহসহ     ʃɔrrahoʃɔho
31998       আধিপত্য    ad̪ʱipot̪t̪o
31999  রাষ্ট্রপতিকে   raʃtropot̪ike

[1000 rows x 2 columns],                    word                     ipa
32000     প্রদর্শনীস্থল       prod̪orʃonist̪ʰɔl
32001         চিত্রশালা            

In [ ]:
# Fills the empty ones
subset1_1000=df.iloc[3000:6000]
for row in subset1_1000.itertuples():
  print(row.Index)
  # print(row[1])
  if pd.notna(row.ipa):
    continue
  print(row.sentence)
  data=call_api(row.sentence)
  # print(data["data"][0])
  if data:
    df.at[row.Index,"ipa"]=data["data"][0]
  else:
    print(row.Index)
    print(row.sentence)
    break
df.to_csv("withIPA.csv",index=False)

**TESTINS**

---



In [3]:
# test connection
def call_api (param):
    url = f"http://127.0.0.1:5000//transcribeBanglaTextToIpa?bangla_ipa={param}"
    response = requests.get (url, verify=False)
    if response.status_code == 200:
        return response.json()
    else:
        return None
data=call_api("আমার সোনার বাংলা।")
data["data"][0]

'amar ʃonar baŋla।'

In [4]:
# Fills the empty ones
subset1_1000=df.iloc[20000:70000]
for row in subset1_1000.itertuples():
  # print(row.Index)
  # print(row[1])
  # if pd.notna(row.ipa):
  #   continue
  # print(row.word)
  data=call_api(row.word)
  # print(data["data"][0])
  if data:
    # df.at[row.Index,"ipa"]=data["data"][0]
    print(f"{row.Index}:{data['data'][0]}")
  else:
    print(row.Index)
    print(row.word)
    break
# df.to_csv("withIPA.csv",index=False)

20000:53
20001:niren
20002:42
20003:surot̪hal
20004:kʰamarer
20005:ɔnner
20006:ʃone
20007:mæsars
20008:ʃeminare
20009:d̪ʱaronapɔt̪ro
20010:aʃik
20011:ikbal
20012:mobarker
20013:ɔst̪robʱandar
20014:andargra͡u̯nder
20015:æt̪od̪ine͡o̯
20016:kolahɔlmɔʲ
20017:nirɟib
20018:hoccʰi
20019:onukulo
20020:mɔʃkor
20021:upokɔntʰe
20022:pʰæʃist
20023:pʰo͡u̯ɟer
20024:minarer
20025:ʃɔŋʃod̪ʱonir
20026:okkʰunno
20027:rohit̪o
20028:bʱæn
20029:læŋɽa
20030:niʲom-nit̪ir
20031:d̪aʃ
20032:pʰa͡i̯nal
20033:ɟi͡e̯m
20034:316
20035:119
20036:kanada
20037:niʃcint̪opur
20038:cʰitu
20039:bæbʃaʲik
20040:pʰircʰilo
20041:d̪ʱɔrmogɔnɟ
20042:meʃi-natker
20043:ipi͡a̯rer
20044:cakrit̪e
20045:ɔpobad̪e
20046:mad̪aripurer
20047:ʃibocre
20048:kiʃorke
20049:balubahi
20050:dubir
20051:cãpat̪la
20052:malopaɽabaʃi
20053:prembag
20054:bãoɽ
20055:mɔt̪ʃnoɟibid̪er
20056:liɟ
20057:proʃɔb
20058:porikkʰati
20059:ʃɔmpad̪oner
20060:ilektronik
20061:dibʱa͡i̯sɟɔnt̪ro
20062:a͡i̯dibi
20063:gɔnoproko͡u̯ʃɔl
20064:onuʃtʰan
20065:kakra͡i̯lo
20066:1

In [ ]:
multiprocessing.cpu_count()

In [43]:
print(pool.shutdown( cancel_futures=True))
df.to_csv(f"withIPA{start_pos}_{end}.csv",index=False)

None


In [ ]:

# w0= pool.submit(subset_worker, subsets[0])
# w1=pool.submit(subset_worker, subsets[1])
# w2=pool.submit(subset_worker, subsets[2])
# w3=pool.submit(subset_worker, subsets[3])
# w4=pool.submit(subset_worker, subsets[4])
# w0.result()
# w1.result()
# w2.result()
# w3.result()
# w4.result()

# print(w0.cancel())
# print(w1.cancel())
# print(w2.cancel())
# print(w3.cancel())
# print(w4.cancel())


In [ ]:
# not working
i=500
steps=100
end=1000
subsets=[]
while i<end:
  subsets.append(df.iloc[i:i+steps])
  i=i+steps
print(subsets)

threads=[]
for subset in subsets:
  threads.append(threading.Thread(target=subset_worker, daemon=True, args=(subset,)))
for thread in threads:
  thread.start();
  time.sleep(1)
for thread in threads:
  thread.join()
print("end")

Cleaning

In [97]:
# Unique word clearing
df=pd.read_csv("bangla_to_ipa_unique_clean2.csv")
df.drop_duplicates(subset=['bangla_token'], inplace=True)
print(len(df))
df.sort_values('bangla_token').to_csv("bangla_to_ipa_unique_clean.csv",index=False)



58569


In [99]:
df=pd.read_csv("bangla_to_ipa_unique_clean2.csv")
remove=[]
for row in df.itertuples():
  if len(row.bangla_token)>2 and row.bangla_token[-2]=="!":
    # print(row.bangla_token[1:])

    for i in range(1,110,1):
      if df.at[row.Index-i,"bangla_token"]==row.bangla_token[:-2]:
        print(row.Index)
        print(row.bangla_token)
        # remove that row
        remove.append(row.Index)
        break

df.drop(remove, inplace=True)
print(len(remove))

0


In [113]:
# check whole dataset
df=pd.read_csv("bangla_to_ipa_unique_clean2.csv")
remove=[]
for row in df.itertuples():
  if row.bangla_token[:-1]=="?":
    # if contains in the whole dataset
    for i in range(len(df)):
      if df.at[i,"bangla_token"]==row.bangla_token[:-1]:
        print(row.Index)
        print(i)
        print(row.bangla_token)
        # remove that row
        remove.append(row.Index)
        break
df.drop(remove, inplace=True)


In [110]:
df.to_csv("bangla_to_ipa_unique_clean2.csv",index=False)

In [83]:
a="bangla_to_ipa_unique_clean"
print(a[1:])

angla_to_ipa_unique_clean
